In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# ResNet50 모델 로드
model= models.resnet50(pretrained=True)

# Conv 레이어 Freeze
for param in model.parameters():
    param.requires_grad = True
    
model.fc = nn.Linear(2048, 6)  # 예시: 6개의 클래스를 분류하는 새로운 출력층

# 하이퍼 파라미터 설정
class Parameters():
    def __init__(self):
        self.description = 'ResNet for Image Classification'
        # 에포크 수
        self.epochs = 50
        # 배치 크기
        self.batch_size = 32
        # 학습률
        self.learning_rate = 1e-4 # 0.0001
        # 훈련된 모델 경로
        self.model_name = 'ResNet'
        # 파일명
        self.data_path = './data/dataset'
args2 = Parameters()

from utils.EarlyStopping import *
from utils.LRScheduler import *
from utils.train_eval_util import train, evaluate


criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(params = model.parameters(),
                             lr = args2.learning_rate)


early_stopping = EarlyStopping(patience = 7,
                               min_delta = 1e-5)


scheduler = LRScheduler(optimizer = optimizer,
                        patience = 5,
                        min_lr = 1e-10,
                        factor = 0.5)

def train(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # 정확도 계산
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total  # 정확도를 퍼센트로 변환

    return avg_loss, accuracy  # 평균 손실과 정확도 반환
best_valid_loss = float('inf')

# 학습률 스케줄러 정의
train_losses = []
valid_losses = []
train_accuracies = []  # 훈련 정확도 기록용 리스트

for epoch in range(args2.epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    valid_loss, valid_acc, _, _ = evaluate(model, valid_loader, criterion, device)

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    train_accuracies.append(train_acc)  # 훈련 정확도 추가

    print(f"Epoch [{epoch+1}/{args2.epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}")

    scheduler(valid_loss)

    if early_stopping(valid_loss):
        break

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), "./" + args2.model_name + ".pth")

# 저장된 모델 로드
import torchvision.models as models


model = models.resnet50().to(device)
model.fc = nn.Linear(2048, 6).to(device)
model.load_state_dict( torch.load( "./" + args2.model_name + ".pth" ) )

# 테스트 세트 평가
test_loss, test_acc, test_preds, test_labels = evaluate(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")